# Load Required Libraries

In [1]:
# For loading plain text documents
from langchain_community.document_loaders import TextLoader

# For splitting large text into smaller chunks (used for embedding)
from langchain_text_splitters import CharacterTextSplitter

# For generating text embeddings using OpenAI
from langchain_openai import OpenAIEmbeddings

# For storing and retrieving embeddings from a vector store/database
from langchain_chroma import Chroma

# For loading environment variables (API keys)
from dotenv import load_dotenv

# For working with data
import pandas as pd


# Load OpenAI API Key from .env

In [2]:
# Load OpenAI API key and any other secrets from the .env file
load_dotenv()

True

# Load Cleaned Book Dataset

In [3]:
# Load the cleaned book data (output from data wrangling)
books = pd.read_csv("books_cleaned.csv")

# Preview the dataset
books


isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [4]:
books['tagged_description']

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

# Export Tagged Book Descriptions to Text File

In [5]:
# Save tagged descriptions (isbn13 + description) to a plain text file
# Each line represents a unique book description for embedding
books['tagged_description'].to_csv(
    "tagged_descriptions.txt",
    sep="\n",
    index=False,
    header=False
)


# Load and Split Text into Chunks

In [8]:
# Load the text file using LangChain's TextLoader
raw_documents = TextLoader("tagged_descriptions.txt").load()

# Instantiate a text splitter
# Each chunk will have up to 1000 characters with no overlap
text_splitter = CharacterTextSplitter(
    chunk_size=0,        # Adjust as needed
    chunk_overlap=0,        # No overlapping chunks
    separator="\n"          # Split at newlines
)

# Split the loaded text into manageable chunks
documents = text_splitter.split_documents(raw_documents)





Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [16]:
documents[0]

Document(metadata={'source': 'tagged_descriptions.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

# create the document embedding and store them in vector database

# Create Document Embeddings and Store in Vector Database


In [28]:
# Create OpenAI embedding model
embedding_model = OpenAIEmbeddings()

# Convert the text chunks into embeddings and store them in a vector database (Chroma)
# This enables fast similarity search
db_books = Chroma.from_documents(documents, embedding=embedding_model)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


# Test Query with Semantic Search

In [33]:
# Test a semantic query to find similar book descriptions
query = 'A book to teach children about nature'

# Retrieve top 10 most semantically similar text chunks
docs = db_books.similarity_search(query, k=10)

# Display the top result (raw chunk of text: includes ISBN and description)
docs[0]


Document(id='87eb5d6f-ddd9-4f58-b41e-247cf87ed2ac', metadata={'source': 'tagged_descriptions.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.')

# Map Back to Original Book Using ISBN


This is important because users want book titles, not raw descriptions, so you map the embedding result back to your original structured data.

In [34]:

# Extract ISBN from the beginning of the chunk (first word in the content)
isbn_result = docs[0].page_content.split()[0].strip()

# Convert to integer and match with original book data
books[books['isbn13'] == int(isbn_result)]


isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [ ]:
# Generalized Retrieval Function for Semantic Recommendations

# Generalized Retrieval Function for Semantic Recommendations


In [35]:
def retrieve_semantic_recommendation(query: str, top_k: int = 10) -> pd.DataFrame:
    """
    Given a natural language query, retrieve top_k semantically similar books
    based on their description embeddings stored in Chroma.

    Args:
        query (str): User's search intent (e.g., "Books about ancient civilizations")
        top_k (int): Number of recommendations to return

    Returns:
        pd.DataFrame: Top k recommended books from the original dataset
    """
    # Perform semantic similarity search on the embedded documents
    recs = db_books.similarity_search(query, k=top_k * 5)  # Fetch more to avoid duplicates

    # Extract the ISBNs from the top-matching documents
    isbn_matches = []
    for doc in recs:
        try:
            isbn = int(doc.page_content.split()[0].strip())
            isbn_matches.append(isbn)
        except ValueError:
            continue  # Skip if the text chunk is malformed

    # Return top_k unique books from the original dataset
    return books[books['isbn13'].isin(isbn_matches)].drop_duplicates('isbn13').head(top_k)



In [37]:
retrieve_semantic_recommendation("Books about personal development for teenagers")



isbn13      isbn10  \
74    9780007195718  0007195710   
266   9780060880125  0060880120   
593   9780140196092  0140196099   
872   9780143039853  0143039857   
1595  9780370307121  0370307127   
1652  9780374528539  0374528535   
1746  9780375829635  0375829636   
1844  9780385481960  0385481969   
1888  9780393040586  0393040585   
1934  9780393703351  0393703355   

                                                  title  \
74    Discover Your Destiny with the Monk Who Sold H...   
266                                   How to Be Popular   
593                                        Who are You?   
872                                       The Outsiders   
1595                                       A Likely Lad   
1652  Get Out of My Life, But First Could You Drive ...   
1746                                     Young Warriors   
1844                                    Youth in Revolt   
1888                                The Confident Child   
1934                         Living on the Razor's Edge   

                            authors              categories  \
74                     Robin Sharma         Conduct of life   
266                       Meg Cabot        Juvenile Fiction   
593                  Malcolm Godwin     Body, Mind & Spirit   
872       S. E. Hinton;Jodi Picoult                 Fiction   
1595                  Gillian Avery       Adventure stories   
1652                Anthony E. Wolf  Family & Relationships   
1746  Tamora Pierce;Josepha Sherman        Juvenile Fiction   
1844                    C. D. Payne                 Fiction   
1888                    T. E. Apter  Family & Relationships   
1934            Matthew D. Selekman              Psychology   

                                              thumbnail  \
74    http://books.google.com/books/content?id=4hVbN...   
266   http://books.google.com/books/content?id=uRsWy...   
593   http://books.google.com/books/content?id=2p8BA...   
872   http://books.google.com/books/content?id=9PyIP...   
1595                                                NaN   
1652  http://books.google.com/books/content?id=oKdbC...   
1746  http://books.google.com/books/content?id=6EnZw...   
1844  http://books.google.com/books/content?id=cZKdO...   
1888  http://books.google.com/books/content?id=stxpQ...   
1934  http://books.google.com/books/content?id=hTe40...   

                                            description  published_year  \
74    A potent pathway to self-awakening that will h...          2004.0   
266   Sixteen-year-old Steph Landry finds an old boo...          2006.0   
593   A noted artist introduces 101 easy-to-follow w...          2000.0   
872   The struggle of three brothers to stay togethe...          1967.0   
1595  Pressured by his father to leave school for a ...          1986.0   
1652  A lighthearted but insightful guide to raising...          2002.0   
1746  Fifteen original short stories by various auth...          2005.0   
1844  Nick Twisp, a dark and precocious fourteen-yea...          1996.0   
1888  Focusing on children from age five to fifteen,...          1997.0   
1934  Employing a multifaceted, flexible approach ba...          2002.0   

      average_rating  num_pages  ratings_count  \
74              3.90      240.0         1956.0   
266             3.54      288.0        19183.0   
593             3.48      224.0          250.0   
872             4.08      160.0         1558.0   
1595            4.08      224.0           12.0   
1652            3.94      240.0         1301.0   
1746            3.91      312.0         3583.0   
1844            4.02      512.0         8158.0   
1888            3.62      270.0           13.0   
1934            4.00      223.0            4.0   

                                     title_and_subtitle  \
74    Discover Your Destiny with the Monk Who Sold H...   
266                                   How to Be Popular   
593            Who are You? 101 Ways of Seeing Yourself   
872                 